# Application de l'algorithme génétique

In [1]:
from reseau_de_neurones import Reseau
import time
import operator
import numpy as np
import pandas as pd
import random
import copy
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

%store -r X_test
%store -r y_test
%store -r X_principalDf
%store -r Y
    
"""
    @retourne un dataframe contenant les exemples d'apprentissage de la deuxième dataset à une haute dimension (titanic)
    Transforme la dimension de l'entrée de la deuxième dataset (10) en une dimension égale à celle de la première dataset (63) 
    en le faisant passer à une couche de réseau de neurones
"""
def conversion_datatest_titanic():
    json_file = open("../TransferLearning/model_titanic.json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # Charger les poids du modèle
    loaded_model.load_weights("../TransferLearning/model_titanic.h5")
    weights = loaded_model.layers[0].get_weights()[0]
    biases = loaded_model.layers[0].get_weights()[1]
    X = X_test_titanic.values
    sortie = []
    for i in range(X_test.shape[0]):
        sortie.extend(np.transpose(np.maximum(np.dot(np.transpose(weights),X[i].reshape(-1,1))+biases.reshape(-1,1), 0)))
    nouvelle_entree = pd.DataFrame(sortie, columns=range(len(sortie[0])))
    return nouvelle_entree

class Algo_gene():
    
    def __init__(self, taille_population, taux_selection, taux_croisement, taux_mutation):
        self.taille_population= taille_population
        self.taux_mutation = taux_mutation
        self.taux_croisement = taux_croisement
        self.taux_selection = taux_selection
        self.population = [Reseau("../TransferLearning/model.json","../TransferLearning/model.h5") for i in range(self.taille_population)]
    
    """
        Générer une population de réseaux différents à partir du réseau entrainé sur les deux dataset
    """
    def generer_population_initial(self):
        for reseau in self.population:
            # Pour chaque couche, modifier un poids wij et un biais bi
            for num_couche in list(reseau.weights.keys()):
                # Modifier un poids wij de la matrice des poids
                num_ligne = np.random.randint(0, len(reseau.weights[num_couche]))
                num_col = np.random.randint(0, len(reseau.weights[num_couche][num_ligne]))
                nouvelle_valeur = random.uniform(-0.2,0.2)
                reseau.weights[num_couche][num_ligne][num_col] = nouvelle_valeur
                # Modifier un poids bi de la matrice des biais
                num_ligne = np.random.randint(0, len(reseau.biases[num_couche]))
                nouvelle_valeur = random.uniform(-0.2,0.2)
                reseau.biases[num_couche][num_ligne] = nouvelle_valeur 
    
    """
        @reseau: le réseau de neurones pour récupérer sa structure, le nombre de couches et de paramètres
        @type_poids: 'biais' ou 'poids' 
        @retourne un tuple repésentant la position aléatoire choisie des matrices du biais ou du poids
        Points où échanger les valeurs du biais ou des poids des parents pour le croisement
    """
    def points_aleatoires(self, reseau, type_poids):
        if type_poids == 'biais':
            couches = list(reseau.biases.keys())
            couche = random.choice(couches)
            points = range(len(reseau.biases[couche]))
            point = random.choice(points)
            return (couche, point)       
        elif type_poids == 'poids':
            couches = list(reseau.weights.keys())
            couche = random.choice(couches)
            num_ligne = np.random.randint(0, len(reseau.weights[couche]))
            num_col = np.random.randint(0, len(reseau.weights[couche][num_ligne]))
            return (couche, num_ligne, num_col)
        
    """
        @retourne la justesse (accuracy) du meilleur individu ou réseau de neurones de la population en fonction de sa justesse
    """
    def meilleur_individu(self):
        return max(self.population, key=lambda reseau: reseau.justesse(X_test_films.values, y_test_films.values, X_test_titanic.values, y_test_titanic.values)).justesse(X_test_films.values, y_test_films.values, X_test_titanic.values, y_test_titanic.values)
    
    """
        @retourne une liste contenant les réseaux de neurones sélectionnés   
        On choisis un pourcentage 'taux_selection' des meilleurs réseau de neurones de la population 
        en fonction de leur accuracy sur les deux dataset
    """
    def selection(self):
        #print("Sélectionner les "+str(int(self.taux_selection * self.taille_population))+" meilleurs solutions")
        self.population.sort(key=lambda reseau: reseau.justesse(X_test_films.values, y_test_films.values, X_test_titanic.values, y_test_titanic.values), reverse=True)
        #mauvais_reseau = random.choice(self.population[int(self.taux_selection * self.taille_population):])
        nouvelle_population = self.population[:int(self.taux_selection * self.taille_population)]
        #nouvelle_population.append(mauvais_reseau)
        return nouvelle_population
    
    """
        @pere et @mere: les deux individus à croiser 
        @retourne deux enfants qui sont deux réseaux de neurones différents générés par la mère et le père 
        Croisement uniforme avec une probabilité 'taux_croisement'
        Les points (à croiser sont choisis aléatoirement par la fonction 'points_aleatoires'
    """
    def croisement(self, pere, mere):
        fils = copy.deepcopy(pere)
        fille = copy.deepcopy(mere)
        # croisement du biais
        for _ in range(sum(len(v)for v in pere.biases.values())):
            couche, point = self.points_aleatoires(mere, 'biais')
            if random.uniform(0,1) < self.taux_croisement:
                fils.biases[couche][point] = mere.biases[couche][point]
                fille.biases[couche][point] = pere.biases[couche][point]
        # croisement des poids
        for _ in range(sum(len(v)for v in pere.weights.values())):
            couche, ligne, colonne = self.points_aleatoires(mere, 'poids')
            if random.uniform(0,1) < self.taux_croisement:
                fils.weights[couche][ligne][colonne] = mere.weights[couche][ligne][colonne]
                fille.weights[couche][ligne][colonne] = pere.weights[couche][ligne][colonne]
        return (fils, fille)
    
    """
        @reseau: l'individu (le réseau de neurones) à muter  
        @retourne le réseau de neurones après mutation 
        Mutation avec une probabilité 'taux_mutation'
        Les points ou gènes à muter sont choisis aléatoirement par la fonction 'points_aleatoires' et remplacés par des valeurs aléatoires
    """
    def mutation(self, reseau):
        reseau_mute = copy.deepcopy(reseau)
        # mutation du biais du reseau
        for _ in range(sum(len(v)for v in reseau.biases.values())):
            couche, point = self.points_aleatoires(reseau, 'biais')
            if random.uniform(0,1) < self.taux_mutation:
                reseau_mute.biases[couche][point] = random.uniform(-0.5,0.5)
        # mutation des poids du reseau
        for _ in range(sum(len(v)for v in reseau.weights.values())):
            couche, ligne, colonne = self.points_aleatoires(reseau, 'poids')
            if random.uniform(0,1) < self.taux_mutation:
                reseau_mute.weights[couche][ligne][colonne] = random.uniform(-0.5,0.5)
        return reseau_mute        

    """
        Algorithme génétique: Sélection, croisement et mutation
    """
    def evolution(self):
        nouvelle_population = self.selection()
        while len(nouvelle_population) < self.taille_population:
            pere = random.choice(nouvelle_population)
            mere = random.choice(nouvelle_population)
            if pere != mere:
                (fils, fille) = self.croisement(pere,mere)
                fils = self.mutation(fils)
                fille = self.mutation(fille)
                nouvelle_population.append(fils)
                if len(nouvelle_population) < self.taille_population:
                    nouvelle_population.append(fille)
        self.population = nouvelle_population
        print("La justesse maximale de l'itération est: {}".format(self.meilleur_individu()))
        
if __name__ == "__main__":
    start = time.time()
    X_test_titanic = X_test
    y_test_titanic = y_test
    # Diviser la dataset des films en train/test
    X_train, X_test_films, y_train, y_test_films = train_test_split(X_principalDf, Y, test_size=0.2, random_state=42)
    # Transformer les features de la dataset test titanic
    X_test_titanic = conversion_datatest_titanic()
    a = Algo_gene(20, 0.2, 0.7, 0.1)
    a.generer_population_initial()
    for i in range(0,1000):
        print("Itération n°: {}".format(i))
        a.evolution()
    end = time.time()
    print("Temps d'exécution: {}s".format(end - start))

Using TensorFlow backend.


Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont chargés
Nombre de couches: 6
Le modèle et les poids sont cha

La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 94
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 95
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 96
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 97
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 98
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 99
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 100
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 101
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 102
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 103
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 104
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 105
La justesse maximale de l'itération est: 83.87799564270153
Itération n°: 106
La ju

La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 201
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 202
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 203
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 204
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 205
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 206
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 207
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 208
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 209
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 210
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 211
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 212
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 213

La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 308
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 309
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 310
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 311
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 312
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 313
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 314
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 315
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 316
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 317
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 318
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 319
La justesse maximale de l'itération est: 84.24110384894699
Itération n°: 320

...

La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 522
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 523
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 524
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 525
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 526
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 527
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 528
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 529
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 530
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 531
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 532
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 533
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 534

La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 629
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 630
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 631
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 632
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 633
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 634
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 635
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 636
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 637
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 638
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 639
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 640
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 641

La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 736
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 737
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 738
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 739
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 740
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 741
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 742
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 743
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 744
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 745
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 746
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 747
La justesse maximale de l'itération est: 84.38634713144518
Itération n°: 748

La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 843
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 844
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 845
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 846
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 847
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 848
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 849
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 850
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 851
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 852
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 853
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 854
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 855

La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 950
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 951
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 952
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 953
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 954
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 955
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 956
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 957
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 958
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 959
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 960
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 961
La justesse maximale de l'itération est: 84.53159041394336
Itération n°: 962